# Chatbot

In [1]:
# pip install pymorphy2

In [3]:
import os
import string
import annoy
import json

from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

import numpy as np
from tqdm import tqdm_notebook
import pandas as pd

### Preprocess the mail.ru responses from the file: attach 1 answer to each question and write it down to a new file, which will be used further.

In [4]:
question = None
written = False

with open("prepared_answers.txt", "w") as fout:
    with open("../input/otvety/Otvety.txt", "r") as fin:
        for line in tqdm_notebook(fin):
            if line.startswith("---"):
                written = False
                continue
            if not written and question is not None:
                fout.write(question.replace("\t", " ").strip() + "\t" + line.replace("\t", " "))
                written = True
                question = None
                continue
            if not written:
                question = line.strip()
                continue


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


0it [00:00, ?it/s]

In [5]:
def preprocess_txt(line):
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

In [6]:
sentences = []

morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation)
c = 0

with open("../input/otvety/Otvety.txt", "r") as fin:
    for line in tqdm_notebook(fin):
        spls = preprocess_txt(line)
        sentences.append(spls)
        c += 1
        if c > 500000:
            break


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if __name__ == '__main__':


0it [00:00, ?it/s]

In [7]:
sentences[:5]

[[],
 [],
 ['вопрос', 'тдв', 'отдыхать', 'лично', 'советовать', 'завести'],
 ['хомячок'],
 ['мужик', 'йопарить', 'собачка', '50', 'кошка']]

In [8]:
# Apply Word2Vec
sentences = [i for i in sentences if len(i) > 2]
model = Word2Vec(sentences=sentences, vector_size=100, min_count=1, window=5)
model.save("w2v_model.model")

In [9]:
# Store just the words + their trained embeddings.
word_vectors = model.wv
word_vectors.save("word2vec.wordvectors")

# Load back with memory-mapping = read-only, shared across processes.
#word_vectors = KeyedVectors.load("../input/models/word2vec.wordvectors", mmap='r')

In [10]:
keys = word_vectors.key_to_index # dictionary of all words in Word2Vec model

In [11]:
len(keys)

572808

In [13]:
index = annoy.AnnoyIndex(100 ,'angular')

index_map = {}
counter = 0

with open("../input/d/irynaalshakova/models/prepared_answers.txt", "r") as f:
    for line in tqdm_notebook(f):
        n_w2v = 0
        spls = line.split("\t")
        index_map[counter] = spls[1]
        question = preprocess_txt(spls[0])
        vector = np.zeros(100)
        for word in question:
            if word in keys.keys():
                vector += word_vectors[word]
                n_w2v += 1
        if n_w2v > 0:
            vector = vector / n_w2v
        index.add_item(counter, vector)
            
        counter += 1

index.build(10)
index.save('speaker.ann')

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  import sys


0it [00:00, ?it/s]

True

In [14]:
with open('index_map.json', 'w') as fp:
    json.dump(index_map, fp)